In [ ]:
import pandas as pd
import numpy as np
import string

In [ ]:
def make_ids(size: float=1e2, length: int=8):
    size = int(size)
    space = string.ascii_lowercase+string.digits
    out = set()
    for _ in range(2*size):
        arr = np.random.choice(list(space), size=length)
        val = ''.join(arr)
        out.add(val)
    return list(out)[:size]

In [ ]:
def make_data(size: float=1e2) -> pd.DataFrame:
    size = int(size)
    years = np.arange(2012, 2017).astype(str)
    df = pd.DataFrame(dict(
        a=np.random.choice(make_ids(8), size=size),
        b=np.random.choice(years, size=size),
        c=np.random.randint(100, size=size)
    ), columns=list('abc'))
    for label in ['a', 'b']:
        df[label] = df[label].astype('category')
    return df

In [ ]:
data = make_data()
data.head()

In [ ]:
lookup = data.groupby(['a', 'b']).mean()

In [ ]:
lookup.loc['bh19zama', '2012']

In [ ]:
lookup_dict = {}
for a in data.a.cat.categories:
    frame = lookup.loc[a].c
    lookup_dict[a] = pd.DataFrame(dict(
        s1=frame.shift(-1),
        s2=frame.shift(-2)
    ), columns=('s1', 's2'))

In [ ]:
for k, tab in lookup_dict.items():
    print(k)
    print(tab)
    break

In [ ]:
out = pd.DataFrame()
for name, group in data.groupby(['a', 'b']):
    a, b = name
    part = lookup_dict[a].loc[b]
    group = group.copy()
    group['s1'] = part.s1
    group['s2'] = part.s2
    group = group.dropna()
    out = out.append(group)

In [ ]:
out